In [ ]:
"""
Classifications and Regressions using features generated from the computed mask

Conduct feature selection with the corresponding classificaton model

Conduct featurer selection with the corresponding regression model

"""

In [2]:
# Project-wide Library calls
import nibabel as nib
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from skimage.util import montage 
from skimage.transform import rotate
from sklearn.model_selection import train_test_split as ttt
from sklearn.metrics import accuracy_score as AS
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm import tqdm
from pylab import *
import gc
import torchmetrics 
import radiomics
from torchvision.utils import save_image
import matplotlib.image
import SimpleITK as sitk
import radiomics



In [3]:
FEAT_PATH ='log/feature_extraction/'
tf=torch.load(FEAT_PATH+'tf.pt')
ef=torch.load(FEAT_PATH+'ef.pt')
ttf=torch.load(FEAT_PATH+'ttf.pt')

ts=torch.load(FEAT_PATH+'ts.pt')
es=torch.load(FEAT_PATH+'es.pt')
tts=torch.load(FEAT_PATH+'tts.pt')

tc=torch.load(FEAT_PATH+'tc.pt')
ec=torch.load(FEAT_PATH+'ec.pt')
ttc=torch.load(FEAT_PATH+'ttc.pt')

"""
# remove features with low variance
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=5)
tf=sel.fit_transform(tf)
ef=sel.fit_transform(ef)
ttf=sel.fit_transform(ttf)

tf=torch.tensor(tf)
ef=torch.tensor(ef)
ttf=torch.tensor(ttf)


"""



# combined all data together
F = torch.cat((tf,ef,ttf))

S = torch.cat((ts,es,tts))

C = torch.cat((tc,ec,ttc))

In [4]:

"""

Classificaton: 

Patients are divided into three classes:

short-term: survival days < 10 months
mid-term: 10 months < survival days < 15 months
long-term: survival days > 15 months

"""


'\n\nClassificaton: \n\nPatients are divided into three classes:\n\nshort-term: survival days < 10 months\nmid-term: 10 months < survival days < 15 months\nlong-term: survival days > 15 months\n\n'

In [4]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel






In [7]:
from sklearn.svm import SVC
# Univariate feature selection
x,xt, y, yt = ttt(F, C, test_size=0.33)

f=SelectKBest(f_classif, k=15)

f.fit(x,y)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)
clf = SVC(kernel = 'poly')

s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=5)
    s+=scores.mean()
    std+=scores.std()
print("Accuracy is: ",s/50, "with a standard deviation of ",std/50)




feature selected: [False False False  True  True  True False False False False False  True
 False False False False False False False  True  True  True False False
 False False False  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
  True False  True  True False  True False False  True False False False
 False False False False False False False False False  True False False
 False False  True]
Accuracy is:  0.3732732732732735 with a standard deviation of  0.04831569598536918


In [65]:
from sklearn.tree import DecisionTreeClassifier as DT
# Univariate feature selection
x,xt, y, yt = ttt(F, C, test_size=0.33)

f=SelectKBest(f_classif, k=15)

f.fit(x,y)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)

clf = DT(max_depth=25)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=5)
    s+=scores.mean()
    std+=scores.std()
print("Accuracy is: ",s/50, "with a standard deviation of ",std/50)


feature selected: [False False  True False False False False  True  True  True False  True
 False False False False False False  True False False False False  True
  True  True False  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True False False False
  True False  True False False False False False  True False False False
 False False False False False False False False False False False False
 False False  True]
Accuracy is:  0.6808558558558557 with a standard deviation of  0.23496433669823794


In [64]:
x,xt, y, yt = ttt(F, C, test_size=0.30)
clf=DT(max_depth=25).fit(x,y)
print(clf.feature_importances_)
selector = SelectFromModel(clf, prefit=True)
print(selector.get_support())
feat=selector.transform(F)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=5)
    s+=scores.mean()
    std+=scores.std()
print("Accuracy is: ",s/50, "with a standard deviation of ",std/50)

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.02086071 0.06079183
 0.01192041 0.         0.07445738 0.         0.         0.02119183
 0.02077556 0.         0.         0.         0.         0.
 0.         0.         0.03932715 0.06599118 0.         0.
 0.08091697 0.         0.         0.         0.         0.
 0.03520549 0.00669216 0.         0.04768162 0.01589387 0.
 0.         0.         0.01192041 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.01589387 0.         0.
 0.01589387 0.         0.         0.02384081 0.         0.
 0.0221379  0.06851806 0.         0.         0.         0.04518084
 0.06349662 0.         0.         0.         0.         0.02582755
 0.         0.         0.         0.02906906 0.         0.
 0.         0.0326005  0.14391434]
[False False False False False False False False False False  True  True
  True False  True False False  Tr

In [9]:
# feature counts =  10
from sklearn.neighbors import KNeighborsClassifier as KNN
# Univariate feature selection
x,xt, y, yt = ttt(F, C, test_size=0.33)

f=SelectKBest(f_classif, k=15)

f.fit(x,y)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)



clf = KNN(n_neighbors=2)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=5)
    s+=scores.mean()
    std+=scores.std()
print("Accuracy is: ",s/50, "with a standard deviation of ",std/50)

feature selected: [False False False False  True  True False False False False False  True
 False False  True  True False False False False  True  True False False
 False False False  True False False  True  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True False False False
 False False False False False False False False False False False False
 False  True False  True False False False False False  True False False
 False False  True]
Accuracy is:  0.42357357357357367 with a standard deviation of  0.10495742520732501


In [10]:
from sklearn.ensemble import RandomForestClassifier
# Univariate feature selection
x,xt, y, yt = ttt(F, C, test_size=0.33)

f=SelectKBest(f_classif, k=15)

f.fit(x,y)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)


clf = RandomForestClassifier(max_depth=30)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=5)
    s+=scores.mean()
    std+=scores.std()
print("Accuracy is: ",s/50, "with a standard deviation of ",std/50)

feature selected: [False False False  True  True  True False  True  True False False  True
 False False False False False False False  True  True  True False  True
  True False False  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True False False False
 False False False False False False False False False  True False False
 False False  True]
Accuracy is:  0.6777747747747748 with a standard deviation of  0.24072901748571546


In [21]:
x,xt, y, yt = ttt(F, C, test_size=0.33)
clf=RandomForestClassifier(max_depth=30).fit(x,y)
print(clf.feature_importances_)
selector = SelectFromModel(clf, prefit=True)
print(selector.get_support())
feat=selector.transform(F)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=5)
    s+=scores.mean()
    std+=scores.std()
print("Accuracy is: ",s/50, "with a standard deviation of ",std/50)

[0.01155833 0.0086185  0.01139438 0.0127226  0.01481829 0.01157132
 0.0148717  0.01130827 0.01083418 0.01035989 0.01198581 0.00631366
 0.00959299 0.00991694 0.00916601 0.00899232 0.0073486  0.00966465
 0.0172773  0.01913691 0.01905914 0.00927756 0.0182518  0.01538171
 0.01459453 0.00900688 0.01279643 0.00478277 0.00477413 0.00937391
 0.01004872 0.01454364 0.00734864 0.01018534 0.00980043 0.00924225
 0.00725117 0.01116929 0.01236379 0.00976193 0.00981912 0.00894058
 0.01749241 0.0065059  0.00387684 0.01003816 0.00943204 0.01001504
 0.0210596  0.00949558 0.00615546 0.00989991 0.01550581 0.00862477
 0.01251178 0.00841423 0.01148931 0.00932934 0.01274869 0.01507952
 0.01497068 0.01042026 0.0071119  0.00736184 0.01168106 0.00638275
 0.01496905 0.01443259 0.00518352 0.00631466 0.01341633 0.00978703
 0.02283965 0.00951877 0.0188071  0.01405535 0.0129138  0.00929329
 0.0098733  0.00714391 0.00706452 0.00950067 0.00988979 0.01990978
 0.01182992 0.00707094 0.04535675]
[ True False False  True  T

In [11]:
# feature count = 10
from sklearn.ensemble import ExtraTreesClassifier
x,xt, y, yt = ttt(F, C, test_size=0.33)

f=SelectKBest(f_classif, k=15)

f.fit(x,y)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)


clf = ExtraTreesClassifier(n_estimators=50)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=5)
    s+=scores.mean()
    std+=scores.std()
print("Accuracy is: ",s/50, "with a standard deviation of ",std/50)

feature selected: [False False False False False False False  True  True  True False  True
 False False False  True False False False False False False False  True
  True  True False  True False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False  True False  True False False False
 False  True False False False False False False False  True False False
 False False  True]
Accuracy is:  0.6593063063063065 with a standard deviation of  0.2538161693464249


In [20]:
x,xt, y, yt = ttt(F, C, test_size=0.33)
clf=ExtraTreesClassifier(n_estimators=50).fit(x,y)
print(clf.feature_importances_)
selector = SelectFromModel(clf, prefit=True)
print(selector.get_support())
feat=selector.transform(F)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=5)
    s+=scores.mean()
    std+=scores.std()
print("Accuracy is: ",s/50, "with a standard deviation of ",std/50)

[0.01383912 0.01078556 0.01079474 0.00892851 0.01307976 0.01050127
 0.00909825 0.01181675 0.00930069 0.00970756 0.01321209 0.01374583
 0.01486045 0.01408376 0.0118671  0.00780581 0.01106186 0.01766824
 0.01748069 0.0150721  0.01292041 0.01100164 0.00937988 0.01099678
 0.01075159 0.00923916 0.01746893 0.0117047  0.01718796 0.01105511
 0.01116672 0.01346429 0.010398   0.00949881 0.01054471 0.0117121
 0.00950127 0.00971607 0.00997502 0.0165067  0.01048018 0.01109899
 0.01192196 0.00856078 0.01174692 0.01119658 0.01213492 0.00886335
 0.0114292  0.01038963 0.00775904 0.01160292 0.0105911  0.01302563
 0.00801934 0.01140157 0.0068646  0.01072894 0.01019424 0.00777793
 0.0078274  0.00735033 0.01035158 0.01097758 0.00955648 0.00916487
 0.01267389 0.01155751 0.01299852 0.00983621 0.01107739 0.00835866
 0.01588588 0.00687991 0.01393564 0.01318748 0.00799918 0.01394524
 0.01080531 0.0081623  0.01247694 0.00697247 0.00732668 0.01748037
 0.0114903  0.00780466 0.0392294 ]
[ True False False False  Tr

In [6]:
"""

Regression part

"""

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,make_scorer

from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import ExtraTreesRegressor as ETR
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.feature_selection import f_regression

In [69]:
# SKlearn feature extraction and regression
# Simple Linear regression select from model
x,xt, y, yt = ttt(F, S, test_size=0.33)
clf=LinearRegression().fit(x,y)

selector = SelectFromModel(clf, prefit=True)
print(selector.get_support())
feat=selector.transform(F)

s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=5)
    s+=scores.mean()
    std+=scores.std()
print("RMSE is: ",sqrt(s/50)," with a standard deviation of ",sqrt(std/50))

[ True False  True False False False False False False False False False
 False False False False  True  True  True False False False False False
 False False False False False False False False False False False  True
 False False False False False False False False False False False False
 False  True False False False False  True False False False False False
 False  True False False False False False False  True False False False
 False  True  True False False False False False False False False False
 False False False]
RMSE is:  349.42849769301887  with a standard deviation of  173.41517626571468


In [83]:
# univariate KNR
from sklearn.neighbors import KNeighborsRegressor as KNR
x,xt, y, yt = ttt(F, S, test_size=0.33)

f=SelectKBest(f_regression, k=20).fit(x,y)

feat=f.transform(F)
# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)
clf=KNR(n_neighbors = 15)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=5)
    s+=scores.mean()
    std+=scores.std()
print("RMSE is: ",sqrt(s/50)," with a standard deviation of ",sqrt(std/50))

feature selected: [False False False  True  True  True False  True  True  True False  True
 False False False  True False False  True  True  True  True False  True
  True  True False  True False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False  True False False  True False False False
 False False  True]
RMSE is:  346.0662147624355  with a standard deviation of  157.40162299171513


In [26]:
# Univariate RFR
f=SelectKBest(f_regression, k=15)

x,xt, y, yt = ttt(F, S, test_size=0.33)
clf=RFR(n_estimators=100,max_depth=40).fit(x,y)
print(clf.feature_importances_)
selector = SelectFromModel(clf, prefit=True)
print(selector.get_support())
feat=selector.transform(F)

s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=5)
    s+=scores.mean()
    std+=scores.std()
print("RMSE is: ",sqrt(s/50)," with a standard deviation of ",sqrt(std/50))

[0.0040313  0.00611878 0.00658284 0.02172046 0.01948598 0.02185816
 0.00711367 0.01201565 0.0058217  0.0075605  0.00562162 0.00933805
 0.01032453 0.01185824 0.013536   0.02376629 0.0031458  0.00454113
 0.00771052 0.02089468 0.00692653 0.02408885 0.01709715 0.01459571
 0.00268371 0.00588545 0.00583516 0.01467167 0.00753224 0.00981241
 0.00901015 0.02136118 0.00177659 0.00265526 0.00452463 0.01351681
 0.00464826 0.00403903 0.00562233 0.00154044 0.00463395 0.00187615
 0.00601049 0.00204542 0.00103833 0.00052668 0.00261713 0.01393707
 0.05318654 0.00445457 0.00274348 0.00588432 0.00734633 0.00243553
 0.00753928 0.00610096 0.00749417 0.00492878 0.00834348 0.00797221
 0.00129804 0.00448988 0.0066865  0.00292373 0.00742919 0.00130314
 0.0366395  0.02185824 0.00197565 0.00178456 0.00250217 0.00572453
 0.03507914 0.01337091 0.02895457 0.02324826 0.01444902 0.0158751
 0.00759491 0.00744567 0.01291554 0.01010054 0.00318064 0.01031647
 0.00662507 0.00631655 0.15792819]
[False False False  True  Tr

In [28]:
# Random Forest regressor select from model

x,xt, y, yt = ttt(F, S, test_size=0.33, random_state = 20)
clf=RFR(n_estimators=100,max_depth=30).fit(x,y)
selector = SelectFromModel(clf,prefit=True)
print(clf.feature_importances_)
print(selector.get_support())
feat=selector.transform(F)
s=0
std=0
for j in range(50):
    scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=5)
    s+=scores.mean()
    std+=scores.std()
print("RMSE is: ",sqrt(s/50)," with a standard deviation of ",sqrt(std/50))

[0.00249943 0.00116055 0.0038227  0.030163   0.01701608 0.0184273
 0.04243279 0.01463645 0.00339122 0.0013705  0.00588829 0.00080308
 0.00678678 0.04437149 0.01172352 0.01070417 0.00534075 0.00197991
 0.00370294 0.01968388 0.025514   0.02321547 0.02408656 0.00709571
 0.00516905 0.00347531 0.00338532 0.00468547 0.00666332 0.04882816
 0.01186935 0.0103876  0.00223202 0.00672415 0.00081525 0.00484547
 0.00446362 0.00600459 0.00887495 0.0037498  0.00411746 0.00218298
 0.00388747 0.00519473 0.00221975 0.00390692 0.00274132 0.00549173
 0.01047959 0.00494643 0.00536699 0.00362994 0.00380871 0.0027111
 0.00996143 0.01271339 0.00753886 0.00531909 0.00708685 0.0079752
 0.001806   0.00840438 0.00108279 0.00240624 0.00320843 0.00159709
 0.03553583 0.0146338  0.00163833 0.00150751 0.00213868 0.01731532
 0.00787578 0.00363532 0.00672161 0.03332773 0.03141678 0.05655911
 0.0046197  0.00711217 0.01248112 0.00519833 0.00359552 0.01110446
 0.01327724 0.00625245 0.13627837]
[False False False  True  True

In [20]:
# Univariate ETR
f=SelectKBest(f_regression, k=15)

x,xt, y, yt = ttt(F, S, test_size=0.33)
clf=ETR(n_estimators = 100).fit(x,y)
print(clf.feature_importances_)
selector = SelectFromModel(clf, prefit=True)
print(selector.get_support())
feat=selector.transform(F)

s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=5)
    s+=scores.mean()
    std+=scores.std()
print("RMSE is: ",sqrt(s/50)," with a standard deviation of ",sqrt(std/50))

[0.00776714 0.00974277 0.01421955 0.00776617 0.01220925 0.00931093
 0.00635536 0.01723769 0.01281077 0.02464712 0.02486255 0.00781511
 0.01945596 0.03675706 0.03166213 0.01257302 0.01215623 0.00784459
 0.01396431 0.00748345 0.01079929 0.01159881 0.00758998 0.02021283
 0.01431729 0.01558544 0.01910749 0.00528404 0.01674065 0.04019659
 0.02315495 0.01198078 0.00406925 0.0049784  0.00411148 0.00677565
 0.00560282 0.00266756 0.00442068 0.00341691 0.00319395 0.004661
 0.00412656 0.00422418 0.00402831 0.00433523 0.00595808 0.0083149
 0.0298925  0.00318965 0.00391693 0.00241753 0.00458434 0.00503731
 0.00968434 0.00863335 0.00721856 0.00985204 0.00685033 0.0068153
 0.0054606  0.00947697 0.0062255  0.00376086 0.0139688  0.0100847
 0.02850303 0.01442664 0.0068795  0.00538473 0.00268723 0.00289878
 0.01223105 0.00482222 0.00902624 0.01930664 0.0062108  0.01365592
 0.00306488 0.03906453 0.01406891 0.01281924 0.00184656 0.0107708
 0.00868321 0.00955258 0.05893267]
[False False  True False  True Fa

In [21]:
# ETR select from model
x,xt, y, yt = ttt(F, S, test_size=0.33)
clf=ETR(n_estimators=100).fit(x,y)
selector = SelectFromModel(clf,prefit=True)
print(clf.feature_importances_)
print(selector.get_support())
feat=selector.transform(F)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=5)
    s+=scores.mean()
    std+=scores.std()
print("RMSE is: ",sqrt(s/50)," with a standard deviation of ",sqrt(std/50))

[0.00604336 0.00593478 0.00412513 0.00926955 0.01037112 0.01340902
 0.01075998 0.01235104 0.00654723 0.00665207 0.01117021 0.00624436
 0.00841798 0.05158663 0.0158639  0.01634612 0.00671204 0.00795732
 0.00621105 0.01169122 0.01292051 0.01308869 0.00708556 0.01701307
 0.00467315 0.00780081 0.00837525 0.00853963 0.00661638 0.04750691
 0.00911366 0.02539257 0.00518463 0.00390898 0.00256584 0.01855814
 0.00583677 0.00587875 0.00316967 0.00747342 0.00648999 0.0049068
 0.00681874 0.00347903 0.00459372 0.00288991 0.0031086  0.02725742
 0.08657217 0.00506393 0.00548095 0.0037059  0.0054467  0.00629322
 0.01947641 0.00465982 0.01361537 0.00678736 0.01143823 0.00758753
 0.01070316 0.00678686 0.00709315 0.00717322 0.00855234 0.00546008
 0.01244683 0.01409111 0.00364777 0.00406802 0.00475315 0.00423705
 0.01227258 0.0084818  0.00775675 0.01723916 0.01480038 0.01140067
 0.00980178 0.00553229 0.01122731 0.0069892  0.00327486 0.00737699
 0.0026137  0.01050134 0.09768013]
[False False False False Fal

In [22]:
from sklearn.tree import DecisionTreeRegressor as DTR

f=SelectKBest(f_regression, k=15)

x,xt, y, yt = ttt(F, S, test_size=0.33)
clf=DTR(criterion = 'friedman_mse',max_depth=15).fit(x,y)
print(clf.feature_importances_)
selector = SelectFromModel(clf, prefit=True)
print(selector.get_support())
feat=selector.transform(F)

s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=5)
    s+=scores.mean()
    std+=scores.std()
print("RMSE is: ",sqrt(s/50)," with a standard deviation of ",sqrt(std/50))

[1.83690019e-04 0.00000000e+00 1.79831073e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.59512319e-01
 1.40882952e-02 5.55348417e-07 0.00000000e+00 3.00929423e-05
 0.00000000e+00 4.62790347e-08 5.29884998e-03 1.10846800e-01
 0.00000000e+00 8.40130414e-06 0.00000000e+00 1.20822990e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 3.23452988e-02
 4.76586882e-03 5.63852189e-05 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 8.40909853e-03 4.02031022e-02
 6.41774514e-03 5.12593720e-03 4.17103626e-02 5.74886020e-04
 0.00000000e+00 3.98856544e-05 2.62110916e-03 5.63123295e-04
 0.00000000e+00 7.05859408e-04 3.80976592e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.75936090e-02 8.41674012e-03
 0.00000000e+00 0.00000000e+00 1.40929858e-03 0.00000000e+00
 4.65916700e-03 4.44741524e-05 9.78426724e-03 2.66659025e-02
 1.88018760e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 7.31411160e-02 9.67182241e-05 0.00000000e+00 3.50387036e-02
 0.00000000e+00 2.830379

In [23]:
x,xt, y, yt = ttt(F, S, test_size=0.33)
clf=DTR(criterion = 'friedman_mse',max_depth=15).fit(x,y)
selector = SelectFromModel(clf,prefit=True)
print(clf.feature_importances_)
print(selector.get_support())
feat=selector.transform(F)
s=0
std=0
for i in range(50):
    scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=5)
    s+=scores.mean()
    std+=scores.std()
print("RMSE is: ",sqrt(s/50)," with a standard deviation of ",sqrt(std/50))

[8.35089784e-06 1.11704968e-03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.07150322e-03 8.42271556e-04
 0.00000000e+00 1.16473754e-02 1.91324375e-04 0.00000000e+00
 0.00000000e+00 1.67067554e-03 2.49428947e-03 2.40302101e-02
 3.34035914e-07 1.27887134e-02 9.31264970e-04 2.85240019e-03
 7.85355548e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.20841090e-02 2.29325344e-02 0.00000000e+00 6.41348954e-05
 3.17392901e-04 9.57080910e-02 0.00000000e+00 0.00000000e+00
 2.80924600e-03 1.20252929e-05 3.55127130e-04 1.88520221e-03
 0.00000000e+00 0.00000000e+00 1.99543157e-04 0.00000000e+00
 1.81404728e-05 0.00000000e+00 6.65536199e-03 3.24682908e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.85456616e-02
 3.72456212e-02 1.52198088e-02 0.00000000e+00 0.00000000e+00
 2.69712884e-04 2.56539582e-04 0.00000000e+00 0.00000000e+00
 7.12030954e-03 1.80886826e-02 0.00000000e+00 0.00000000e+00
 1.52936974e-02 3.74565605e-04 1.90029320e-05 2.85099189e-04
 0.00000000e+00 2.589574